# Step 1: Prep Data

In [1]:
import pandas as pd
import dtale

# Import dataframes

# ach_df with names of bank entities
ach_df = pd.read_csv('FedACHDir.csv')

# fednow_df with the routing numbers of banks that use FedNow
fednow_df = pd.read_csv('Fednow_RTNs.csv')

# rtp_df with the routing numbers of banks that use RTP
rtp_df = pd.read_csv('RTP_RTNs.csv')

In [2]:
# Replace Routing_Number in ach_df with New_RN if New_RN is non-zero
ach_df['Routing_Number'] = ach_df.apply(lambda row: row['New_RN'] if row['New_RN'] != 0 else row['Routing_Number'], axis=1)

# Check work
dtale.show(ach_df)

In [3]:
# Drop unnecessary New_RN column
ach_df_no_new_rn = ach_df.drop(columns=['New_RN'])

# Remove duplicates based on routing number
ach_df_no_duplicates = ach_df_no_new_rn.drop_duplicates(subset='Routing_Number', keep='first')

# Drop other unnecessary columns
ach_df_cleaned = ach_df_no_duplicates.drop(columns=['Office_Code','Rec_Type','Change_Date','Mail_Address','Zip_EXT','Area_Code','Prefix','Suffix','Inst_Status','Data_View'])

print(ach_df_cleaned)

       Routing_Number  FRB_Number                        Customer_Name  \
0            11000015    11000015                 FEDERAL RESERVE BANK   
1            11000028    11000015  STATE STREET BANK AND TRUST COMPANY   
2            11000138    11000015                BANK OF AMERICA, N.A.   
3            11000206    11000015                  BANK OF AMERICA N.A   
4            11000390    11000015                  BANK OF AMERICA N.A   
...               ...         ...                                  ...   
17222       325272306   121000374                          TONGASS FCU   
17223       325272335   121000374                 MATANUSKA VALLEY FCU   
17224       325272351   121000374         TONGASS FEDERAL CREDIT UNION   
17226       325280039   121000374             MAC FEDERAL CREDIT UNION   
17227       655060042    51000033       SOCIAL SECURITY ADMINISTRATION   

            City State    Zip  
0        ATLANTA    GA  30309  
1      N. QUINCY    MA   2171  
2        HENRIC

# Step 2: Flag FedNow and RTP Customers

### 1 = yes
### 0 = no

In [4]:
# Create a copy of the ACH data to add flags
df_with_flags = ach_df_cleaned.copy()

# Ensure 'Routing_Number' is the same data type across all datasets (convert to string)
df_with_flags['Routing_Number'] = df_with_flags['Routing_Number'].astype(str)
fednow_df['Routing_Number'] = fednow_df['Routing_Number'].astype(str)
rtp_df['Routing_Number'] = rtp_df['Routing_Number'].astype(str)

In [5]:
# Merge with FedNow dataset
df_with_flags = df_with_flags.merge(fednow_df[['Routing_Number']], on='Routing_Number', how='left', indicator='in_fednow')

# Create the FedNow flag column: 1 if the routing number is found in fednow_df, otherwise 0
df_with_flags['in_fednow'] = (df_with_flags['in_fednow'] == 'both').astype(int)

# Merge with RTP dataset
df_with_flags = df_with_flags.merge(rtp_df[['Routing_Number']], on='Routing_Number', how='left', indicator='in_rtp')

# Create the RTP flag column: 1 if the routing number is found in rtp_df, otherwise 0
df_with_flags['in_rtp'] = (df_with_flags['in_rtp'] == 'both').astype(int)

# Fill NaN values for missing IDs (from merging) with 0 for both flags
df_with_flags['in_fednow'] = df_with_flags['in_fednow'].fillna(0).astype(int)
df_with_flags['in_rtp'] = df_with_flags['in_rtp'].fillna(0).astype(int)

# Final output: Display the DataFrame with the flags
dtale.show(df_with_flags)

# Save the DataFrame with the flags
df_with_flags.to_csv('FedNow_RTP_in_dir.csv', index=False)

# Step 3: Check for Unidentified Routing Numbers and Extract to Separate Datasets

In [6]:
# Check to see how many of the routing numbers were identified in ach_df

num_rows_fednow = fednow_df.shape[0]

num_rows_rtp = rtp_df.shape[0]

# Print the number of rows
print(f"Number of banks in FedNow: {num_rows_fednow}")
print(f"Number of banks in RTP: {num_rows_rtp}")

Number of banks in FedNow: 1425
Number of banks in RTP: 1744


In [7]:
# Count the number of 1's in the 'in_fednow' and 'in_rtp' columns
count_in_fednow = df_with_flags['in_fednow'].sum()
count_in_rtp = df_with_flags['in_rtp'].sum()

# Print the counts
print(f"Number of 1's in 'in_fednow': {count_in_fednow}")
print(f"Number of 1's in 'in_rtp': {count_in_rtp}")

Number of 1's in 'in_fednow': 1407
Number of 1's in 'in_rtp': 1634


In [8]:
# Identify unmatched routing numbers for FedNow
unmatched_fednow = fednow_df[~fednow_df['Routing_Number'].isin(ach_df['Routing_Number'])]

# Identify unmatched routing numbers for RTP
unmatched_rtp = rtp_df[~rtp_df['Routing_Number'].isin(ach_df['Routing_Number'])]

In [9]:
# Save unmatched FedNow routing numbers
unmatched_fednow.to_csv('Unmatched_Fednow_RTNs.csv', index=False)

# Save unmatched RTP routing numbers
unmatched_rtp.to_csv('Unmatched_RTP_RTNs.csv', index=False)